In [ ]:
from IPython.display import Image
from IPython.core.display import HTML

As we know, multiplying two matrices $A(m \times n)$ and $B(n \times p)$ results in matrix $AB(m \times p)$.
- Complexity of computing AB is $O(mnp)$

Matrix multiplication is associative:

$ABC = (AB)C = A(BC)$

Order gives the same result but, affects the complexity of computing. 

`Example`: Suppose we have matrices $A(1,100)$, $B(100,1)$ and $C(1,100)$. 
- Computing $BC(100, 100)$ will be $100 \times 1 \times 100$ and computing $A(BC)$ will then be $1 \times 100 \times 100$. Totalling 20000 steps. 

On the other hand
- Computing $AB(1, 1)$ will be $1 \times 100 \times 1$ and computing $(AB)C$ will then be $1 \times 1 \times 100$. Totalling 200 steps.


## Optimal way to multiply n such matrices, choose $k$

That is find bracketing structure, whether $(AB)C$ or $A(BC)$ for three matrices. Or for four matrices:

![Representation as a tree](./imgs/matMul-2025-01-09-2201.png)

 For n such matrices we need to find k such that $(M_1 \times M_2 \times M_3 \times \cdots \times M_k) \times (M_{k+1} \times M_{k+2} \times M_{k+3} \times \cdots \times M_n)$ for some $1 \leq k < n$. FIrst factor will have dim $(r_1,c_k)$ and other will have $(r_{k+1},c_n)$ with the cost $O(r_1c_kc_n)$.

### Subproblems
- Final step $(M_1 \times M_2 \times M_3 \times \cdots \times M_k) \times (M_{k+1} \times M_{k+2} \times M_{k+3} \times \cdots \times M_n)$

- Subproblem are $(M_1 \times M_2 \times M_3 \times \cdots \times M_k)$ and $(M_{k+1} \times M_{k+2} \times M_{k+3} \times \cdots \times M_n)$

- Total cost = Cost of Subproblem 1 + Cost of subproblem + Cost of final problem which is $(r_1c_kc_n)$

- $n-1$ choices of $k$. Yet again we have $j$ for each subproblem $1 \cdots k$. So to `generalize`:

$Cost(M_i \times M_{i+1} \times  \cdots \times M_j) = min(Cost(M_i \times M_{i+1} \times \cdots \times M_k) + Cost(M_{k+1} \times \cdots \times M_j) + (r_i c_k c_j)$

## Final Equation

- Base case (No multiplication) : $Cost(i,i) = 0$ 
- $Cost(i,j) = min_{i \leq k < j} ([Cost(i,k) + Cost(k+1,j) + (r_i c_k c_j)])$ only when $i \leq j$